In [ ]:
using Plots
using ImageView
using FFTW
using Serialization

## Schema

In [ ]:
function bytesToString(file, start, stop)
    len = stop-start+1
    seek(file,start-1)
    String(read(file,len))
end

function bytesToUInt32(file, start, stop)
    #TODO check if range matches size
    seek(file, start-1)
    Int32(ntoh(read(file,UInt32)))
end

function bytesToUInt16(file, start, stop)
    #TODO check if range matches size
    seek(file, start-1)
    Int16(ntoh(read(file,UInt16)))
end

function bytesToArray(file, start, stop)
    len = stop-start+1
    seek(file, start-1)
    read(file,len)
end

#Image File Descriptor
imageFileDescrictorScheme = [    
        ("length", (9, 12), bytesToUInt32),
        ("numSignals", (181, 186), bytesToString),
        ("sarDataBytes", (187, 192), bytesToString),
        ("bitsPerSample", (217, 220), bytesToString),
        ("samplesPerPixel", (221, 224), bytesToString), #could be 1 or 2 (IQ or A)
        #245-272 - borders and interleaving
        ("physicalRecordsPerLine", (273, 274), bytesToString),
        #281-400 - unexplored
        ("sarDataFormat", (401, 428), bytesToString),
        ("sarDataFormatTypeCode", (429,432), bytesToString)
        
        ]

signalDataRecordScheme = [
        ("recordSequenceNumber", (1,4), bytesToUInt32),
        ("lengthOfRecord", (9,12), bytesToUInt32),
        ("sarImageDatalineNumber", (13, 16), bytesToUInt32),
        ("pixelCount", (25, 28), bytesToUInt32),
        #33-56 - instrument settings
        #57-84 - chirp characteristics
        ("PRF (mHz)", (57,60), bytesToUInt32),
        ("Chirp type", (67,68), bytesToUInt16),    #0 for linear
        
        #93-388 - platform info - important!!!
        ("Valid", (97,100),bytesToUInt32),
        ("Electronic Squint 1",(101,104),bytesToUInt32),
        ("Electronic Squint 2",(109,112),bytesToUInt32),
        ("Mechanical Squint",(113,116),bytesToUInt32),
        ("FirstSampleRange (m)",(117,120),bytesToUInt32),
        ("Platform Altitude (m)",(141,144),bytesToUInt32),
        ("Platform Ground Speed (cm/s)",(145,148),bytesToUInt32),
        ("signalData", (413, 22110), bytesToArray)
        ]

datasetSummaryRecordScheme = [
    ("wavelength", (501,516), bytesToString),
    ("chirpType", (519,534), bytesToString),
    ("coeffs",(534,694),bytesToString),
    ("samplingRate",(711,726), bytesToString),
    ("pulseLength",(743,758),bytesToString),      #us
    ("PRF", (935,950), bytesToString),            #mHz
    ("doppler", (1415,1654), bytesToString)
]

function parseFile(file,scheme,offset=0)
    keyMap = Dict([])
    for i=1:length(scheme)
        keyMap[scheme[i][1]] = i  #make dictionary mapping field names to indices
    end
    results = []
    for (name, (start, stop), f) in scheme
        len  = stop-start+1
        push!(results, f(file,start+offset,stop+offset))
    end
    return (key=keyMap, fields=results)
end

# FFTs

In [ ]:
#https://dsp.stackexchange.com/a/38546
function complexFFT(I,Q)
    Ifreq = FFTW.fft(I)
    Qfreq = FFTW.fft(Q)
    
    resultI = real(Ifreq)-imag(Qfreq)
    resultQ = real(Qfreq)+imag(Ifreq)
    resultI, resultQ
end

In [ ]:
FFTW.fft(Complex{Float32}.([1,2,3,4,5,6,7,8,9,0],[1,0,1,0,1,0,1,0,1,0]))

In [ ]:
FFTW.fft(Complex{Float16}.([1,2,3,4,5,6,7,8,9,0],[1,0,1,0,1,0,1,0,1,0]))

In [ ]:
complexFFT([1,2,3,4,5,6,7,8,9,0],[1,0,1,0,1,0,1,0,1,0])

# File Processing

In [ ]:
loadFromSerialized = false
signalRecords = []

if loadFromSerialized
    signalRecords = Serialization.deserialize(open("signalRecords2.ser","r"))
    print("Loaded signalRecords")
else
    file = open("ALPSRP271180830-L1.0/IMG-HH-ALPSRP271180830-H1.0__A")

    imageDescriptor = parseFile(file,imageFileDescrictorScheme)
    sarDataBytes    = parse(Int64,imageDescriptor.fields[imageDescriptor.key["sarDataBytes"]])
    numSignals      = parse(Int64,imageDescriptor.fields[imageDescriptor.key["numSignals"]])

    signalRecords = [ ]

    for i = 1:(numSignals-1)
        parsedRecord = parseFile(file,signalDataRecordScheme,720+21100*i)
        signal = parsedRecord.fields[parsedRecord.key["signalData"]]

        #TODO  - is 15 or 16 better?? compute mean of signals?
        #TODO  - DEAL WITH VALUES > 0x1f!!
        signal = map(x->min(0x1f,x), signal)

        signal = Int8.(signal)-Int8.(15*ones(size(signal)))
        I = signal[1:2:length(signal)]
        Q = signal[2:2:length(signal)]

        parsedRecord.key["I"] = length(parsedRecord.fields)+1;
        parsedRecord.key["Q"] = length(parsedRecord.fields)+1;
        push!(parsedRecord.fields, I)
        push!(parsedRecord.fields, Q)

        parsedRecord.fields[parsedRecord.key["signalData"]] = []  #remove original signalData from record!

        push!(signalRecords,parsedRecord)
        if(i%10000==0)
            print(i)
            print("   ")
            print(Base.summarysize(parsedRecord))
            print("B    ")
            print(Base.summarysize(signalRecords)/1e6)
            println(" MB")
        end
    end
    print(Base.summarysize(signalRecords)/1e6)
    println(" MB")

    close(file)
    Serialization.serialize(open("signalRecords2.ser","w"),signalRecords)
end
        

In [ ]:
#Serialization.serialize(open("/home/thatcher/Documents/12.421/signalRecords2.ser","w"),signalRecords)

In [ ]:
#signalRecords = Serialization.deserialize(open("/home/thatcher/Documents/12.421/signalRecords2.ser","r"))
#print("Loaded signalRecords")

In [ ]:
#=
file = open("ALPSRP271180830-L1.0/IMG-HH-ALPSRP271180830-H1.0__A")

imageDescriptor = parseFile(file,imageFileDescrictorScheme)
sarDataBytes    = parse(Int64,imageDescriptor.fields[imageDescriptor.key["sarDataBytes"]])
numSignals      = parse(Int64,imageDescriptor.fields[imageDescriptor.key["numSignals"]])

signalRecords = [ ]

for i = 1:(numSignals-1)
    parsedRecord = parseFile(file,signalDataRecordScheme,720+21100*i)
    signal = parsedRecord.fields[parsedRecord.key["signalData"]]
    
    #TODO  - is 15 or 16 better?? compute mean of signals?
    #TODO  - DEAL WITH VALUES > 0x1f!!
    signal = map(x->min(0x1f,x), signal)
    
    signal = Int8.(signal)-Int8.(15*ones(size(signal)))
    I = signal[1:2:length(signal)]
    Q = signal[2:2:length(signal)]

    parsedRecord.key["I"] = length(parsedRecord.fields)+1;
    parsedRecord.key["Q"] = length(parsedRecord.fields)+1;
    push!(parsedRecord.fields, I)
    push!(parsedRecord.fields, Q)
    
    parsedRecord.fields[parsedRecord.key["signalData"]] = []  #remove original signalData from record!
    
    push!(signalRecords,parsedRecord)
    if(i%10000==0)
        print(i)
        print("   ")
        print(Base.summarysize(parsedRecord))
        print("B    ")
        print(Base.summarysize(signalRecords)/1e6)
        println(" MB")
    end
end
print(Base.summarysize(signalRecords)/1e6)
println(" MB")

close(file)
=#

In [ ]:
rec = signalRecords[110]
rec.fields[rec.key["Mechanical Squint"]]

Size of original dictionary based-implementation:

5811778483 bytes = 5811 MB
Now down to 

## TODO:
Rewrite without dictionaries. signalRecords takes up 5.8e9 bytes which is wayyy too big! like 8 times too big. Maybe with tuples (named tuples?) this could be much smaller. (and faster)

# Chirp Finding
Found a nice clean signal on signal 25850 at pixel 3395

In [ ]:
#process header
file = open("ALPSRP271180830-L1.0/LED-ALPSRP271180830-H1.0__A")
rec = parseFile(file,datasetSummaryRecordScheme,720)
#rec.fields[rec.key["PRF"]]

f = 1.2700000E+09
fdot = -1.0370370E+12
sampleRate = 32e6     #Hz
pulseLength = 27e-6   #s
#pulseSamples = Integer(floor(pulseLength*sampleRate))
pulseSamples = 80
print("Max IF freq: ")
println(fdot*pulseLength)
Sif(t) = exp(-2*pi*im*(fdot*t)*t)
t = 1/sampleRate* (range(1, stop = pulseSamples) |> collect)

sig = Sif.(t)/sqrt(pulseSamples)

rangeCells = 10000

chirpI = vcat(real(sig),zeros(rangeCells-length(sig)))
chirpQ = vcat(imag(sig),zeros(rangeCells-length(sig)))

chirpFFTI, chirpFFTQ = complexFFT(chirpI,chirpQ)
complexChirpFFT = Complex.(chirpFFTI,chirpFFTQ)

close(file)
print("Ready")

### Old Approach
Find what looks like a pulse in the image

In [ ]:
start = 3395     #pixels, 1 indexed
len = 80      #pixels
rec = signalRecords[25850]
I = rec.fields[rec.key["I"]]
Q = rec.fields[rec.key["Q"]]
chirpIfrag = I[start:1:start+len]
chirpQfrag = Q[start:1:start+len]
plot(chirpIfrag)
plot!(chirpQfrag)
#chirpI = vcat(chirpIfrag,zeros(length(I)-length(chirpIfrag)))
#chirpQ = vcat(chirpQfrag,zeros(length(Q)-length(chirpQfrag)))

# Image Forming

In [ ]:
showRaw = false;
loadSmallSignalsFromSerialized = false

if loadSmallSignalsFromSerialized
    smallSignals = Serialization.deserialize(open("smallSignals.ser","r"))
    smallSignals = smallSignals'
    print("Loaded smallSignals")
else
    #sub image formation:
    signalRecords = Serialization.deserialize(open("signalRecords2.ser","r"))
    println("Loaded signalRecords")

    sampleNum = 10000 #how many samples of each echo to keep
    echoNum = 5000  #how many echos to keep
    echostart = 1

    smallSignals = zeros(Complex{Float16},sampleNum,echoNum)

    #This is soooooo much faster than hcatS!!!!
    #https://stackoverflow.com/questions/38308053/julia-how-to-fill-a-matrix-row-by-row-in-julia
    for i = echostart+1:echostart+echoNum
        rec = signalRecords[i]
        I = Float16.(rec.fields[rec.key["I"]])
        Q = Float16.(rec.fields[rec.key["Q"]])

        smallSignals[:,i-echostart] = Complex.(I[1:sampleNum],Q[1:sampleNum])
    end

    println("Done")
end

In [ ]:
signalRecords = []

In [ ]:
#Serialization.serialize(open("smallSignalsBigger.ser","w"),smallSignals)

In [ ]:
shape = size(smallSignals)

pixelsPerStrip = shape[1]

cimg = zeros(Complex{Float16},shape)
if showRaw
    img =  zeros(Float64,shape)
end

for i = 1:shape[2]
    
    mag = abs.(smallSignals[:,i])
    I = real.(smallSignals[:,i])
    Q = imag.(smallSignals[:,i])
    ####### Range Compression
    
    signalFFTI, signalFFTQ = complexFFT(I,Q)
    complexSignalFFT = Complex.(signalFFTI,signalFFTQ)
    #print("#")
    crossCorrelated = FFTW.ifft(conj.(complexChirpFFT).*complexSignalFFT)
    I = real(crossCorrelated)
    Q = imag(crossCorrelated)
    
    ####### End Range Compression
    
    complex = Complex.(I,Q)
    cimg[:, i] = complex
    if showRaw
        img[:, i]  = mag
    end
    if i%1000 == 1
        println(i)
    end
end

if showRaw
    img = img'
end
cimg= cimg'

print("Done")

In [ ]:

img = []

### Range Cell Migration Correction
Going to using the Range Doppler Algorithm. Take range compressed data, fourier transform along each line of constant range, then interpolate by a azimuth-frequency-dependent amount to correct for range cell migration.

Could image at this point, but probably won't look different. 

After that, do azimuth compression as usual. To be efficient, don't apply IFFT to RCM corrected data and instead use that direction in the azimuth convolutions.

Range shift at each frequency is:
$$\Delta R(f_n) = \frac{\lambda^2 R_0 f_n^2}{8 V_r^2}$$
Where $R_0$ is the distance of closest approach, $V_r$ is the effective radar velocity (Cummings and Wong pg. 235)


In [ ]:
rcmc = zeros(Complex{Float16},size(cimg))
Vr = vorbital

rcmcft = zeros(Complex{Float16},size(cimg))

for i = 1:size(cimg)[2]
    line = cimg[:,i]
    lineI = real(line)
    lineQ = imag(line)

    ####### Fourier Transforming

    lineFFTI, lineFFTQ = complexFFT(lineI,lineQ)
    complexLineFFT = Complex.(lineFFTI,lineFFTQ)

    rcmcft[:,i] = complexLineFFT
    if i%1000 == 1
        print("#")
    end
end


2

In [ ]:
#now we want to shift each frequency in range space, so make each frequency bin a column for speed
rcmcft = rcmcft'
rcmc = rcmc'
shape = size(rcmcft)

rangeCellLength = 3e8/sampleRate * 0.75

for i = 1:shape[2]
    n = i
    #the "highest frequencies" are actually the negative frequencies aliased up!
    if n>shape[2]/2
        n = shape[2]-i
    end
    fn = n-1/shape[2]*PRF    #check this but pretty sure
    
    #range migration distance in meters
    ΔR = wavelength^2*R0*fn^2/(8*Vr^2)
    c = 3e8
    cellshift = Integer(round(ΔR/rangeCellLength))
    #cellshift = Integer(round(ΔR/c))
    
    #interpolation
    #NEAREST NEIGHBOR - bad!
    rcmc[:,i] = vcat(rcmcft[cellshift+1:shape[1],i],zeros(Complex{Float64},cellshift))
    
    if n%1000==0
        print(n)
        print("  ")
        println(cellshift)
    end
end
rcmcft = rcmcft'
shifted = rcmc'
2

### Azimuth Compression!!
need PRF, altitude, initial range, ground speed, wavelength
$$R(s) = \sqrt{R_0^2+s^2v^2} = R_0+\dot{R}_0s+\frac{1}{2}\ddot{R}_0s^2$$
$$C(s) = e^{i \frac{4\pi}{\lambda} R(s)}$$ (4pi comes from: phase shift due to distance is 2pi, but you go there and back so phase shift is doubled)
Sample $C(s)$ at $s=n/PRF$

For zero doppler shift (kinda naive case but fine), $\dot{R}_0 = 0$ and $\ddot{R}_0 = \frac{v^2}{R_0}$

**Note:** this all only works for the first pixels!! Need to correct $R_0$ as we move out from the ground track.

In [ ]:
R0 = 848965       #m
PRF = 2141.3276231263  #Hz

a = (6371+628)*1000.0  #m            #https://www.eorc.jaxa.jp/ALOS-2/en/about/overview.htm
G = 6.67408e-11
Me= 5.97219e24       #kg
P = sqrt(4*pi^2/(G*Me)*a^3)
print("Period: ")
print(P/3600)
println(" h")
vorbital = 2pi*a/P            #These are terrible assumptions!!!!!!!
vground = 2pi*6371000.0/P     #These are terrible assumptions!!!!!!!!
println("Orbital Velocity: ",vorbital," m/s")
println("Ground~~ Velocity: ",vground," m/s")


wavelength = 0.2360571 #m

R(s) = R0 + 1/2*vorbital^2/R0*s^2
C(s) = exp(-4pi*im/wavelength*R(s))

width = 400
s = 1/PRF*(range(-width, stop = width) |> collect)

sig = C.(s)/sqrt(width)

azimuthI = vcat(real(sig),zeros(size(cimg)[1]-length(sig)))
azimuthQ = vcat(imag(sig),zeros(size(cimg)[1]-length(sig)))

azimuthFFTI, azimuthFFTQ = complexFFT(azimuthI,azimuthQ)
complexAzimuthFFT = Complex.(azimuthFFTI,azimuthFFTQ)

print("Ready")

In [ ]:
azcomp = zeros(Float16,size(cimg))

for i = 1:size(cimg)[2]
    line = cimg[:,i]
    lineI = real(line)
    lineQ = imag(line)
    
    ####### Azimuth Compression
    
    #lineFFTI, lineFFTQ = complexFFT(lineI,lineQ)
    lineFFTI = real(shifted[:,i])
    lineFFTQ = imag(shifted[:,i])
    complexLineFFT = Complex.(lineFFTI,lineFFTQ)
    crossCorrelated = FFTW.ifft(conj.(complexAzimuthFFT).*complexLineFFT)
    I = real(crossCorrelated)
    Q = imag(crossCorrelated)
    
    ####### End Azimuth Compression
    
    
    #complex = Complex.(I,Q)
    result = abs.(I.^2+Q.^2)
    azcomp[:,i] = result
    if i%1000 == 1
        print("#")
    end
end
azcomp = azcomp'
2

## Images

In [ ]:
#magnitude image formation
scaling = 10
shape = size(azcomp)
#raw = abs.(view(transpose(img),
#                1:scaling:5000,
#                1:scaling:8000))
#rangecompmag = (abs.(view(transpose(cimg),
#                1:scaling:5000,
#                1:scaling:8000)))
#rcmcftmag = (abs.(view(rcmc,
#                1:scaling:5000,
#                1:scaling:8000)))
azcompmag = (abs.(view(azcomp,
                1:scaling:shape[1],
                1:scaling:shape[2])))
#reduced = zeros(pixelsPerStrip)
#for c=1:(length(reduced)-2)                 #TODO why is -2 here??
#    reduced[c] = sum(mag[c*pixelStep:(c+1)*pixelStep])/pixelStep
#end
print("Ready")

In [ ]:
imshow(azcompmag)

In [ ]:
imshow(raw)

In [ ]:
imshow(rangecompmag)

In [ ]:
imshow(rcmcftmag)

## Image Notes:
There are a few interesting features: one is very long saturated strips that fill out most of an entire signal data record. This might have to be removed.... not sure why they are present at all.

The other feature is short dashes that are ~67 pixels long in the vertical direction. I suspect the pulse length is 67 sampling intervals long.